In [1]:
import json
import openai
import requests

import dotenv

dotenv.load_dotenv()

client = openai.OpenAI()

BASE_URL = "https://nomad-movies.nomadcoders.workers.dev"

In [16]:
# 실제 API를 호출하는 함수들

def get_popular_movies():
    """인기 영화 목록을 가져옵니다."""
    response = requests.get(f"{BASE_URL}/movies")
    return response.json()

def get_movie_details(movie_id: int):
    """특정 영화의 상세 정보를 가져옵니다."""
    response = requests.get(f"{BASE_URL}/movies/{movie_id}")
    return response.json()

def get_movie_credits(movie_id: int):
    """특정 영화의 출연진 및 제작진 정보를 가져옵니다."""
    response = requests.get(f"{BASE_URL}/movies/{movie_id}/credits")
    return response.json()

In [17]:
# OpenAI function calling에 사용할 tools 정의

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_popular_movies",
            "description": "현재 인기 있는 영화 목록을 가져옵니다. 사용자가 인기 영화, 최신 영화, 요즘 뜨는 영화 등을 물어볼 때 사용합니다.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_details",
            "description": "특정 영화의 상세 정보(제목, 줄거리, 장르, 평점, 개봉일, 런타임 등)를 가져옵니다. 영화 ID를 알고 있을 때 해당 영화에 대한 정보를 조회합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "movie_id": {
                        "type": "integer",
                        "description": "조회할 영화의 ID",
                    }
                },
                "required": ["movie_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_credits",
            "description": "특정 영화의 출연진(배우) 및 제작진 정보를 가져옵니다. 영화에 누가 출연하는지, 감독이 누구인지 등을 물어볼 때 사용합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "movie_id": {
                        "type": "integer",
                        "description": "출연진을 조회할 영화의 ID",
                    }
                },
                "required": ["movie_id"],
            },
        },
    },
]

In [19]:
# 함수 이름 → 실제 함수 매핑
available_functions = {
    "get_popular_movies": get_popular_movies,
    "get_movie_details": get_movie_details,
    "get_movie_credits": get_movie_credits,
}

SYSTEM_PROMPT = """당신은 영화 전문가 에이전트입니다.
사용자의 질문에 대해 적절한 함수를 호출하여 영화 정보를 제공합니다.
답변은 한국어로 친절하게 해주세요."""


def ask_movie_agent(user_message: str):
    """Movie Expert Agent: 사용자 질문을 받아 function calling으로 처리합니다."""
    print(f"🎬 사용자: {user_message}\n")

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_message},
    ]

    # 1단계: LLM에게 어떤 함수를 호출할지 결정하게 함
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
    )

    message = response.choices[0].message

    # tool_calls가 없으면 바로 답변 반환
    if not message.tool_calls:
        print(f"🤖 에이전트: {message.content}")
        return

    # 2단계: 모델이 선택한 함수 확인 및 실행
    messages.append(message)

    for tool_call in message.tool_calls:
        fn_name = tool_call.function.name
        fn_args = json.loads(tool_call.function.arguments)

        print(f"⚙️  호출 함수: {fn_name}")
        print(f"📋 인자: {fn_args}\n")

        # 실제 함수 실행
        fn = available_functions[fn_name]
        result = fn(**fn_args)

        # 함수 실행 결과를 메시지에 추가
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(result, ensure_ascii=False),
        })

    # 3단계: 함수 결과를 포함하여 최종 답변 생성
    final_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )

    print(f"🤖 에이전트: {final_response.choices[0].message.content}")

In [20]:
# 테스트 1: 인기 영화 조회
ask_movie_agent("지금 인기 있는 영화가 무엇인지 알려줘")

🎬 사용자: 지금 인기 있는 영화가 무엇인지 알려줘

⚙️  호출 함수: get_popular_movies
📋 인자: {}

🤖 에이전트: 현재 인기 있는 영화들을 소개해 드리겠습니다.

1. **Mercy**  
   - 개요: 미래의 한 형사가 아내를 살해한 혐의로 재판을 받으며, 그가 사랑했던 고도의 AI 판사에게 무죄를 입증하기 위해 90분의 시간을 가지게 됩니다.
   - 평점: 7.1  
   - 개봉일: 2026-01-20  
   ![Mercy](https://image.tmdb.org/t/p/w780/pyok1kZJCfyuFapYXzHcy7BLlQa.jpg)

2. **28 Years Later: The Bone Temple**  
   - 개요: 의사 켈슨이 세상을 바꿀 수 있는 충격적 관계에 휘말리게 되고, 그 과정에서 겪는 고난을 이야기합니다.
   - 평점: 7.2  
   - 개봉일: 2026-01-14  
   ![28 Years Later: The Bone Temple](https://image.tmdb.org/t/p/w780/kK1BGkG3KAvWB0WMV1DfOx9yTMZ.jpg)

3. **The Orphans**  
   - 개요: 두 친구가 고아원에서 함께 자랐으나, 얽히고설킨 사건으로 다시 만나게 되면서 일어나는 이야기입니다.
   - 평점: 6.0  
   - 개봉일: 2025-08-20  
   ![The Orphans](https://image.tmdb.org/t/p/w780/hP7mjZr2SVfjAorlRHTdV1XZmHY.jpg)

4. **A Woman Scorned**  
   - 개요: 여동생의 복수를 위해 두 자매가 격렬한 사투에 나서는 내용의 스릴러입니다.
   - 평점: 6.0  
   - 개봉일: 2025-06-09  
   ![A Woman Scorned](https://image.tmdb.org/t/p/w780/dlOSBiNULMPzKIze84LDjvEN9z1.jpg)

5. **Zootopia 

In [21]:
# 테스트 2: 영화 상세 정보 조회
ask_movie_agent("movie ID 550에 해당하는 영화가 무엇인지 알려줘")

🎬 사용자: movie ID 550에 해당하는 영화가 무엇인지 알려줘

⚙️  호출 함수: get_movie_details
📋 인자: {'movie_id': 550}

🤖 에이전트: 영화 ID 550에 해당하는 영화는 **"파이트 클럽" (Fight Club)**입니다. 이 영화는 1999년 10월 15일에 개봉하였으며, 장르는 드라마와 스릴러입니다.

- **감독:** 데이비드 핀처
- **출연:** 에드워드 노튼, 브래드 피트, 헬레나 본햄 카터 등
- **런타임:** 139분
- **예산:** 63,000,000달러
- **수익:** 100,853,753달러
- **평점:** 8.4/10 (31493명의 평가)

**영화 개요:** 
불면증에 시달리는 주인공과 미끄러운 비누 판매자가 원초적인 남성 공격성을 통해 충격적이고 새로운 형태의 치료를 창안합니다. 이들의 개념은 각 마을마다 underground '파이트 클럽'을 형성하게 되고, 불가사의한 인물이 등장하면서 통제가 불가능한 혼돈으로 치닫는 이야기입니다.

![포스터](https://image.tmdb.org/t/p/w780/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg)

더 많은 정보는 [여기](http://www.foxmovies.com/movies/fight-club)에서 확인하실 수 있습니다.


In [22]:
# 테스트 3: 영화 출연진 조회
ask_movie_agent("movie ID 550에 해당하는 영화에 누가 출연하는지 알려줘")

🎬 사용자: movie ID 550에 해당하는 영화에 누가 출연하는지 알려줘

⚙️  호출 함수: get_movie_credits
📋 인자: {'movie_id': 550}

🤖 에이전트: 영화 ID 550에 해당하는 영화는 **"파이트 클럽"**입니다. 이 영화에 출연한 주요 배우들은 다음과 같습니다:

1. **에드워드 노튼 (Edward Norton)** - 역할: Narrator  
   ![에드워드 노튼](https://image.tmdb.org/t/p/w185/8nytsqL59SFJTVYVrN72k6qkGgJ.jpg)

2. **브래드 피트 (Brad Pitt)** - 역할: Tyler Durden  
   ![브래드 피트](https://image.tmdb.org/t/p/w185/cckcYc2v0yh1tc9QjRelptcOBko.jpg)

3. **헬레나 본햄 카터 (Helena Bonham Carter)** - 역할: Marla Singer  
   ![헬레나 본햄 카터](https://image.tmdb.org/t/p/w185/hJMbNSPJ2PCahsP3rNEU39C8GWU.jpg)

4. **미트 로프 (Meat Loaf)** - 역할: Robert Paulson  
   ![미트 로프](https://image.tmdb.org/t/p/w185/7gKLR1u46OB8WJ6m06LemNBCMx6.jpg)

5. **자레드 레토 (Jared Leto)** - 역할: Angel Face  
   ![자레드 레토](https://image.tmdb.org/t/p/w185/ca3x0OfIKbJppZh8S1Alx3GfUZO.jpg)

이 외에도 많은 배우들이 출연했습니다. 더 궁금한 점이 있으면 언제든지 질문해 주세요!
